# 0 Information

## 0.1 Prerequisites

### 0.1.1 Git
* Should be installed
* Should be available via path, so that it is possible to call `git clone ...` directly from the console

### 0.1.2 Python
It is recommended to use version 3.8+. The oldest version on which
this notebook and all codebase scripts have been tested is 3.7.

## 0.2 Recommendations

### 0.2.1 Configuration
The current setup of this file allow you to completely run all the modules of the project without any problems.
If you wish to simply execute all the functions, and you don't require special configuration or output directories
then please stick to the configuration that is provided here by default.

### 0.2.2 Virtual environment
It is recommended to run this notebook in a separate `conda` python virtual environment.
Even though this is not a requirement to run this notebook it is very recommended using a virtual environment.

### 0.2.3 Google colab
A lot of scripts from here are very slow and it is advisable to run them from google colab. In this you don't have to care about virtual environments and packages in python

# 1 Setup and config

### `SCRIPTS_EXEC_MODE`
It is possible to execute all scripts in `verbose` mode to display
all logs that are displayed by the scripts themselves
and also by all the subprocesses that they start. However, if you use `silent` mode
then no logs will be displayed. Only important messages like errors and
important execution checkpoints will be logged in silent mode.

In [8]:
SCRIPTS_EXEC_MODE = 'verbose'

### `CLONE_GITHUB_REPO`

Set to true if this is a standalone notebook that gets executed and needs to pull the code from the repo. Otherwise we assume this notebook is a sibling to all the root files of the repo

In [9]:
CLONE_GITHUB_REPO = False

### `EVAL_SPLIT_RATIO`
Specify a float value from `0.0` to `1.0` which defines how big the part of images should
be used for evaluation. Value of `0.2` means that 20% of all the images in the provided dataset
should be used for evaluation. The rest is used for training

In [10]:
EVAL_SPLIT_RATIO = 0.2

### `NUM_TRAIN_STEPS`
The number of training steps during the model training process. A training step is ...TODO...

In [11]:
NUM_TRAIN_STEPS = 100

### `CHECKPOINT_EVERY_N`
How often should Tensorflow make checkpoints of training status

In [12]:
CHECKPOINT_EVERY_N = 10

### `EXISTING_MODEL`

If you do not need to train the model, please specify the name of the trained model here and put its weights in the model

In [13]:
EXISING_MODEL = None

# 2 Workspace preparation
All of these cells should be run only for one time when we "start from scratch" with this notebook
Install the project package to the python (virtual) environment

In [14]:
if CLONE_GITHUB_REPO:
    !git clone -b animal-detection "https://github.com/Maks0bs/study-project-cv-real-vs-virtual.git" ./animal-detection
    %cd animal-detection

In [16]:
import pip

pip.main(['install', '-e', '.'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Obtaining file:///D:/Programming/Projects/python/ksunia/animal-breed-detection/studienprojekt_cv_rvv

Preparing metadata (setup.py): started

Preparing metadata (setup.py): finished with status 'done'

Installing collected packages: studienprojekt-cv-rvv

...

Uninstalling studienprojekt-cv-rvv-1.0:

Successfully uninstalled studienprojekt-cv-rvv-1.0

Running setup.py develop for studienprojekt-cv-rvv

Successfully installed studienprojekt-cv-rvv-1.0

0

In [17]:
! python -m pip install matplotlib pillow gin-config tensorflow==2.13.0 opencv-python

## 2.1 Optional: upload archive with data

If executing on colab, you need to upload the archive from [here](https://www.kaggle.com/datasets/zippyz/cats-and-dogs-breeds-classification-oxford-dataset/data) to the `data/datasets/raw/my_dataset` folder and unzip it

In [18]:
#TODO

# 3 TFOD API and pretrained model
Here we install Tensorflow Object Detection API and
load the pretrained model that is specified in the config.
Generally, the TFOD API should only be installed once.
However, several pretrained models can be loaded (when the config is changed)
and in this case it is required to run these cells once again
for the new pretrained model to be used during training.

In [19]:
import os
python_file = os.path.join('setup', 'workspace.py')
! python {python_file} -m {SCRIPTS_EXEC_MODE} -v "both"

Verifying if TFOD API is installed (correctly)...
TFOD API is installed (correctly)
Pretrained model was loaded (correctly)


## 4 Dataset preparation
Here we create the processed dataset with splitting into train and test sets.
Please put your raw perception data in ./data/datasets/raw/my_dataset

In [20]:
import os
python_file = os.path.join('data', 'preparation.py')
!python {python_file} -m {SCRIPTS_EXEC_MODE} -sr {EVAL_SPLIT_RATIO}

Reading raw dataset
Successfully read raw dataset
Writing processed dataset
Successfully wrote processed dataset
generate tf records from d:\programming\projects\python\ksunia\animal-breed-detection\studienprojekt_cv_rvv/data/datasets/processed\my_dataset\train
records written successfully
generate tf records from d:\programming\projects\python\ksunia\animal-breed-detection\studienprojekt_cv_rvv/data/datasets/processed\my_dataset\eval
records written successfully


## 5 Train the model
Train the model on the created TF records

In [ ]:
!python {os.path.join(MAIN_PACKAGE_PATH, 'train', 'train.py')} -m {SCRIPTS_EXEC_MODE} --num_train_steps {NUM_TRAIN_STEPS} --checkpoint_every_n {CHECKPOINT_EVERY_N}

## 6 Evaluate the model

In [7]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format('/content/studienprojekt_cv_rvv/tfod_api/object_detection/model_main_tf2.py', '/content/studienprojekt_cv_rvv/models/trained/my_model' ,'/content/studienprojekt_cv_rvv/models/trained/my_model/pipeline.config', '/content/studienprojekt_cv_rvv/models/trained/my_model')
command

'python /content/studienprojekt_cv_rvv/tfod_api/object_detection/model_main_tf2.py --model_dir=/content/studienprojekt_cv_rvv/models/trained/my_model --pipeline_config_path=/content/studienprojekt_cv_rvv/models/trained/my_model/pipeline.config --checkpoint_dir=/content/studienprojekt_cv_rvv/models/trained/my_model'

In [ ]:
!{command}

## 7 Image Object Detection

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Image Object Detection

PIPELINE_CONFIG_FILE = ''
LABELMAP_FILE = ''
NEW_CKPT_PATH = ''

configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_FILE)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(NEW_CKPT_PATH, 'ckpt-XXX')).expect_partial()


@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(LABELMAP_FILE)
IMAGE_PATH = ''

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=10,
            min_score_thresh=.25,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.savefig('test.png')
plt.show()